In [1]:
#IMPORTING LIBRARIES
from nsepy import get_history
#to get publicly available data on the current NSEIndia site by communicating with their REST APIs.
from datetime import date, datetime  
# supplies classes for manipulating dates and times.
import os 
#provides functions for interacting with the operating system. 
from tradingview_ta import TA_Handler, Interval, Exchange 
#unofficial Python API wrapper to retrieve technical analysis from TradingView
import time 
#time module allows to work with time in Python.
from selenium import webdriver
#selenium represents a set of api's that can be used to control a browser programmatically.
#webdriver used for the validation of websites (and web applications)
from selenium.webdriver.chrome.service import Service
#Returns us an instance of Chrome driver through which we will be interacting with Chrome browser.
#service will open chrome browser
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
#os.system("figlet -c Smart Algo Trading Bot ")
Today = date.today()
y = Today.strftime("%Y")
m = Today.strftime("%m")
d = Today.strftime("%d")
# d = "30"

In [3]:
#last order
last_order="sell"
sold_before = False
bought_before = False
current_price = 0
take_profit = 0.0
take_loss = 0.0

In [4]:
#load chrome driver 
# driver = webdriver.Chrome(executable_path="/Users/mrm/Downloads/chromedriver")
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get("https://in.tradingview.com/")
time.sleep(60)

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:02<00:00, 2.85MB/s]


In [5]:
#initiating tradingview handler to get the recomendation for sonata software for 15 min interval
ssw = TA_Handler(
    symbol="RELIANCE",
    screener="india",
    exchange="NSE",
    interval=Interval.INTERVAL_5_MINUTES
)
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timer = '{:02d}:{:02d}'.format(mins, secs)
        print(timer, end="\r")
        time.sleep(1)
        t -= 1
while True:
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    if(current_time >= "09:30:00" and current_time < "15:10:00"):

        rec = ssw.get_analysis()
        RSI = rec.indicators["RSI"]
        # MACD = rec.indicators["MACD.macd"]
        EMA = rec.moving_averages["COMPUTE"]["EMA10"]
        print("RSI:", RSI, "EMA:", EMA)


        if ( RSI >= 30 and RSI <= 70 and EMA == "BUY" ):
            if (last_order=="sell"):
                print("Buying 1 stock of RELIANCE")
                last_order="buy"
                print(last_order)
                print(sold_before)
                #buy 1 stock of RELIANCE
                driver.find_element(By.XPATH,"//div[2]/div[2]/div[1]/div/div/div[1]/div[2]/div/div[2]").click()
                driver.find_element(By.XPATH,"//div[2]/div[2]/div[1]/div/div/div[1]/div[6]/button/div/span[2]").click()
                current_price = driver.find_element(By.XPATH,"/html/body/div[2]/div[1]/div[2]/div[1]/div/table/tr[1]/td[2]/div/div[2]/div[1]/div[2]/div[1]").text
                print(current_price)
                take_profit = float(current_price) + 8
                take_loss = float(current_price) - 5
                while True:
                    print("Time left till next call - ")
                    countdown(int(5))
                    rec = ssw.get_analysis()
                    RSI = rec.indicators["RSI"]
                    # MACD = rec.indicators["MACD.macd"]
                    EMA = rec.moving_averages["COMPUTE"]["EMA10"]
                    print("RSI:", RSI, "EMA:", EMA)
                    current_price = driver.find_element(By.XPATH,"//div[2]/div[1]/div[2]/div[1]/div/table/tr[1]/td[2]/div/div[2]/div[1]/div[2]/div[1]").text
                    if((RSI >= 30 and EMA == "SELL") or (float(current_price) >= take_profit) or (float(current_price) <= take_loss)):
                        #sell the stock
                        print("Selling 1 stock of RELIANCE")
                        last_order="sell"
                        print(last_order)
                        #sell 1 stock of RELIANCE 
                        driver.find_element(By.XPATH,"//div[2]/div[2]/div[1]/div/div/div[1]/div[2]/div/div[1]").click()
                        time.sleep(2)
                        driver.find_element(By.XPATH,"//button[1]/div[1]/span[2]").click()
                        break
                    else:
                        print("no adjustment required")
            else:
                print("last order not sold")
        elif( RSI >= 50 and EMA == "SELL" ):
            if ( last_order == "sell"):
                print("selling stock of  RELIANCE")
                driver.find_element(By.XPATH,"//div[2]/div[2]/div[1]/div/div/div[1]/div[2]/div/div[1]").click()
                time.sleep(2)
                driver.find_element(By.XPATH,"//button[1]/div[1]/span[2]").click            ()
                current_price = driver.find_element(By.XPATH,"//div[2]/div[1]/div[2]/div[1]/div/table/tr[1]/td[2]/div/div[2]/div[1]/div[2]/div[1]").text
                print(current_price)
                take_profit = float(current_price) - 8
                take_loss = float(current_price) + 5
                while True:
                    print("Time left till next call - ")
                    countdown(int(5))
                    rec = ssw.get_analysis()
                    RSI = rec.indicators["RSI"]
                    EMA = rec.moving_averages["COMPUTE"]["EMA10"]
                    print("RSI:", RSI, "EMA:", EMA)
                    current_price = driver.find_element(By.XPATH,"//div[2]/div[1]/div[2]/div[1]/div/table/tr[1]/td[2]/div/div[2]/div[1]/div[2]/div[1]").text
                    if((RSI <= 30 and EMA == "BUY") or ( float(current_price) <= take_profit) or (float(current_price) >= take_loss)):
                        #buy the stock
                        print("Buying the stock")
                        driver.find_element(By.XPATH,"//div[2]/div[2]/div[1]/div/div/div[1]/div[2]/div/div[2]").click()
                        driver.find_element(By.XPATH,"//div[2]/div[2]/div[1]/div/div/div[1]/div[6]/button/div/span[2]").click()
                        break
                    else:
                        print("no adjustment required")

        else:
            print("condition not favourable..waiting")
          
    elif(current_time >= "15:10:00"):
        print("Time to close for the day")
        # #fetch open profit
        open_profit = driver.find_element(By.XPATH,"//div[2]/div[7]/div[2]/div[4]/div/div[1]/div/div[2]/div[3]/div[1]").text
        # print(open_profit)
        # P = "1000"
        print("Calculating profit :",open_profit)
        break
    else:
        if(current_time >= "09:15:00" and current_time < "09:30:00"):
            print("Analysing market","\n\n")
        elif(current_time < "9:15:00"):
            print("Waiting for market to open")
        else:
            print("No action required")
        

Time to close for the day


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0080F243]
	(No symbol) [0x00797FD1]
	(No symbol) [0x0068D04D]
	(No symbol) [0x00672D7A]
	(No symbol) [0x006DBE7B]
	(No symbol) [0x006EC196]
	(No symbol) [0x006D8386]
	(No symbol) [0x006B163C]
	(No symbol) [0x006B269D]
	GetHandleVerifier [0x00AA9A22+2655074]
	GetHandleVerifier [0x00A9CA24+2601828]
	GetHandleVerifier [0x008B8C0A+619850]
	GetHandleVerifier [0x008B7830+614768]
	(No symbol) [0x007A05FC]
	(No symbol) [0x007A5968]
	(No symbol) [0x007A5A55]
	(No symbol) [0x007B051B]
	BaseThreadInitThunk [0x75E200F9+25]
	RtlGetAppContainerNamedObjectPath [0x77B87BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77B87B8E+238]
